In [ ]:
pip install -U openai

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [ ]:
import pandas as pd
from datetime import datetime,timedelta
import azure.cosmos.cosmos_client as cosmos_client
import azure.cosmos.exceptions as exceptions
from azure.cosmos.http_constants import StatusCodes
from azure.cosmos.partition_key import PartitionKey
from datetime import *
from azure.cosmos import exceptions, CosmosClient, PartitionKey,ProxyConfiguration
from pyspark.sql.functions import concat,col,lit


In [ ]:
from openai import OpenAI
clientGPT = OpenAI(api_key='{secret_key}')

## Read sa.articles

In [ ]:
import datetime

current_time = datetime.datetime.now()
timestamp = int(current_time.timestamp())
print(timestamp)

1713523696


In [ ]:
## Lay data trong 1 tieng gan nhat
query = f"WHERE c._ts >= {timestamp} - 3600"

In [ ]:
query

'WHERE c._ts >= 1713523696 - 100400'

In [ ]:
# Config Authentic CosmosDB
from azure.cosmos import CosmosClient
CONNECTION_STRING = "{connection_string}"
client = CosmosClient.from_connection_string(CONNECTION_STRING)
database_name = "sa"
container_name = "articles"
database = client.get_database_client(database_name)
container = database.get_container_client(container_name)

In [ ]:
items = list(container.query_items(
        query=f"SELECT c.article, c.content, c.created_date FROM c {query} ",
        enable_cross_partition_query=True
    ))
items

[{'article': 'https://cafef.vn/truong-dai-hoc-duoc-vi-nhu-cung-dien-chau-au-doc-nhat-vo-nhi-o-viet-nam-hoc-phi-re-bat-ngo-lum-xum-kien-tung-giua-co-dong-gop-von-la-2-anh-em-ruot-188240324221805058.chn',
  'content': 'Tại Việt Nam, thị trường giáo dục, đặc biệt phân khúc trường đại học thu hút rất nhiều “đại gia” trong ngoài nước tham gia đầu tư. Các trường thu hút sinh viên với nhiều mô hình, chương trình giảng dạy cho đến kiến trúc xây dựng. Trong đó, trường Đại học Võ Trường Toản tại tỉnh Hậu Giang nhiều năm qua vẫn là ngôi trường gây chú ý nhất.Trường đại học được ví như cung điện độc nhất vô nhị ở Việt NamTheo giới thiệu, Đại học Võ Trường Toản là ngôi trường đại học tư thục đầu tiên của tỉnh Hậu Giang. Trường được thành lập vào tháng 2/2008, xây dựng theo mô hình “Thành phố đại học” mang phong cách châu Âu trên diện tích 20 hecta.Trường có thể đáp ứng nhu cầu học tập của trên 18.000 sinh viên, với đầy đủ cơ sở vật chất như khu giảng đường lý thuyết, khu thực hành thí nghiệm, nhà h

In [ ]:
df = pd.DataFrame(items)
print(df)

                                             article  ... created_date
0  https://cafef.vn/truong-dai-hoc-duoc-vi-nhu-cu...  ...   2024-04-18
1  https://cafef.vn/doanh-nghiep-ti-hon-ganh-ga-k...  ...   2024-04-18

[2 rows x 3 columns]


In [ ]:
document_text = ""
prompt = """
Identify each stock code mentioned in the document.
Summary the information mentioned about each stock code.
Each stock code's summary must be less than 200 characters.
The summary must be in Vietnamese.
Use strictly only information from the document. Do not generate extra information.
Each stock code must be summarized only one time.
Evaluate the sentiment regarding each stock code.

<<Document>>
{text}
<<Document>>

Instructions:
 - List stocks, must appear in the text

"""

Instruction = """
Returned results must be in Vietnamese.
Returned results must include: StockID, Content, Sentiment.
StockID: a unique code for each stock, composed of only three capital letters, must appear in the text, esspecially remember that each stock code must be summarized only one time.
Content: A brief summary related to the stock, with a maximum of 200 characters
Sentiment: positive, negative, neutral
Your response should be in JSON format with the following mandatory fields.
{
"StockID":{
  "content": "Summary information related with the stock in a short sentence of max 200 characters",
  "sentiment": "evaluate the status of stock via content summary, choose one of positive, negative, neutral"
}
}
"""
text = prompt.format(text = document_text) + Instruction

In [ ]:
def labeling(text_to_analyze = ""):
  response = clientGPT.completions.create(
    model= "gpt-3.5-turbo-instruct",
    prompt=text,
    temperature=0,
    max_tokens=700,
    top_p=0.7,
    frequency_penalty=0,
    presence_penalty=0
  )
  return response


In [ ]:
df

,article,content,created_date
0,https://cafef.vn/truong-dai-hoc-duoc-vi-nhu-cu...,"Tại Việt Nam, thị trường giáo dục, đặc biệt ph...",2024-04-18
1,https://cafef.vn/doanh-nghiep-ti-hon-ganh-ga-k...,"Ngày 3/4 tới đây, CTCP Cảng Xanh Vip (mã VGR) ...",2024-04-18


In [ ]:
results = []
for i in range(len(df)):
    try:
        document_text = df.loc[i, 'content']
        text = prompt.format(text=document_text) + Instruction
        result = labeling(document_text).choices[0].text.strip()
        results.append((df.loc[i, 'article'], df.loc[i, 'created_date'],df.loc[i, 'content'],result))
    except Exception as e:
        print(f"Error processing row {i}: {e}")
        document_text = df.loc[i, 'content'][:3000]
        text = prompt.format(text=document_text) + Instruction
        result = labeling(document_text).choices[0].text.strip()
        results.append((df.loc[i, 'article'], df.loc[i, 'created_date'],df.loc[i, 'content'],result))

df_result = pd.DataFrame(results, columns=['article', 'created_date', 'content','result'])


In [ ]:
df_result.display()

article created_date content result https://cafef.vn/truong-dai-hoc-duoc-vi-nhu-cung-dien-chau-au-doc-nhat-vo-nhi-o-viet-nam-hoc-phi-re-bat-ngo-lum-xum-kien-tung-giua-co-dong-gop-von-la-2-anh-em-ruot-188240324221805058.chn 2024-04-18 Tại Việt Nam, thị trường giáo dục, đặc biệt phân khúc trường đại học thu hút rất nhiều “đại gia” trong ngoài nước tham gia đầu tư. Các trường thu hút sinh viên với nhiều mô hình, chương trình giảng dạy cho đến kiến trúc xây dựng. Trong đó, trường Đại học Võ Trường Toản tại tỉnh Hậu Giang nhiều năm qua vẫn là ngôi trường gây chú ý nhất.Trường đại học được ví như cung điện độc nhất vô nhị ở Việt NamTheo giới thiệu, Đại học Võ Trường Toản là ngôi trường đại học tư thục đầu tiên của tỉnh Hậu Giang. Trường được thành lập vào tháng 2/2008, xây dựng theo mô hình “Thành phố đại học” mang phong cách châu Âu trên diện tích 20 hecta.Trường có thể đáp ứng nhu cầu học tập của trên 18.000 sinh viên, với đầy đủ cơ sở vật chất như khu giảng đường lý thuyết, khu thực hành thí nghiệm, nhà hiệu bộ, văn phòng làm việc, khu trung tâm thương mại - giải trí và ký túc xá.Bên cạnh đó, trường có hệ thống kiến trúc hiện đại, với hàng trăm mái vòm, cột tháp, mang đậm phong cách châu Âu. Màu xanh của cây cối kết hợp với màu sắc chủ đạo trong thiết kế gồm trắng, xám và đỏ bordeaux giúp tôn lên vẻ đẹp huyền bí và cổ tích của ngôi trường này.Trường còn có cả Khu công viên Kittyd & Minnied được thiết kế theo phong cách châu Âu với nhiều tòa nhà, lâu đài và nhiều trò chơi giải trí hấp dẫn, như tàu lượn siêu tốc, vòng quay ngựa gỗ, xe điện đụng….Ảnh: Một số hình ảnh của Trường Đại học Võ Trường Toản.Học phí “rẻ nhất Việt Nam”, chỉ từ 5,25 triệu/học kỳXịn sò là vậy, song trường lại có mức học phí rẻ đến bất ngờ, cam kết “rẻ nhất Việt Nam”. Theo Thông tin tuyển sinh niên khoá 2023 - 2024, học phí của Võ Trường Toản là 5,25 - 5,7 triệu/học kỳ với các ngành đang “hot” như: Tài chính - Ngân hàng, Quản trị Kinh doanh, Kế toán, Kinh doanh Quốc tế, Thương mại điện tử, Quản lý Nhà nước, Luật, Công nghệ thông tin.Trường cũng đào tạo các ngành y khoa với học phí 30,75 triệu đồng/học kỳ, dược với 21,25 triệu đồng/học kỳ, răng hàm mặt với 30,75 triệu/học kỳ. Một năm có 2 học kỳ.Trường còn cung cấp các gói học bổng cho sinh viên điểm cao. {
"VTT":{
 "content": "Trường Đại học Võ Trường Toản tại tỉnh Hậu Giang là ngôi trường gây chú ý nhất với hệ thống kiến trúc hiện đại và học phí rẻ nhất Việt Nam.",
 "sentiment": "positive"
},
"VNM":{
 "content": "Thị trường giáo dục, đặc biệt phân khúc trường đại học tại Việt Nam thu hút nhiều đại gia trong và ngoài nước tham gia đầu tư.",
 "sentiment": "neutral"
},
"CTG":{
 "content": "Trường Đại học Võ Trường Toản có hệ thống kiến trúc hiện đại và học phí rẻ nhất Việt Nam, đào tạo nhiều ngành hot như Tài chính - Ngân hàng, Quản trị Kinh doanh, Kế toán, Kinh doanh Quốc tế, Thương mại điện tử, Quản lý Nhà nước, Luật, Công nghệ thông tin.",
 "sentiment": "positive"
},
"VRE":{
 "content": "Trường Đại học Võ Trường Toản có hệ thống kiến trúc hiện đại và học phí rẻ nhất Việt Nam, đào tạo nhiều ngành hot như Tài chính - Ngân hàng, Quản trị Kinh doanh, Kế toán, Kinh doanh Quốc tế, Thương mại điện tử, Quản lý Nhà nước, Luật, Công nghệ thông tin.",
 "sentiment": "positive"
},
"VIC":{
 "content": "Trường Đại học Võ Trường Toản có hệ thống kiến trúc hiện đại và học phí rẻ nhất Việt Nam, đào tạo nhiều ngành hot như Tài chính - Ngân hàng, Quản trị Kinh doanh, Kế toán, Kinh doanh Quốc tế, Thương mại điện tử, Quản lý Nhà nước, Luật, Công nghệ thông tin.",
 "sentiment": "positive"
},
"HPG":{
 "content": "Trường Đại học Võ Trường Toản có hệ thống kiến trúc hiện đại và học phí rẻ nhất Việt Nam, đào tạo nhiều ngành hot như Tài chính - Ngân hàng, Quản trị Kinh doanh, Kế toán, Kinh doanh Quốc tế, Thương mại điện tử, Quản lý Nhà nước, Luật, Công nghệ thông tin https://cafef.vn/doanh-nghiep-ti-hon-ganh-ga-khong-lo-nganh-cang-bien-loi-nhuan-moi-nam-dong-gop-200-300-ty-co-tuc-son-son-lai-hay-choi-lon-co-phieu-boc-dau-len-dinh-1882403

In [ ]:
import json
data_list = []

for index, row in df_result.iterrows():
    try:
        result_str = row['result']
        if not result_str.endswith("}\n}"):
            last_brace_index = result_str.rfind('}')
            result_str = result_str[:last_brace_index+1] + "\n}"
            result_json = json.loads(result_str)
        else:
            result_json = json.loads(result_str)

        for stock_code, content_sentiment in result_json.items():
            content = content_sentiment['content']
            sentiment = content_sentiment['sentiment']
            data_list.append({'article': row['article'], 'created_date': row['created_date'], 'content_detail': row['content'], 'stockCode': stock_code, 'content': content, 'label': sentiment})
    except Exception as e:
        print(f"Error processing row {index}: {e}")

df_stock = pd.DataFrame(data_list)
stock_filtered_df = df_stock[df_stock.apply(lambda row: f"{row['stockCode']}" in row['content_detail'], axis=1)]
stock_filtered_df = stock_filtered_df[["article", "created_date", "stockCode", "content", "label"]]
# df_new = df_stock[["url", "lastmod", "stockCode", "content", "sentiment"]]
stock_filtered_df

,article,created_date,stockCode,content,label
5,https://cafef.vn/doanh-nghiep-ti-hon-ganh-ga-k...,2024-04-18,VGR,VGR sẽ chốt danh sách cổ đông nhận cổ tức đợt ...,positive
6,https://cafef.vn/doanh-nghiep-ti-hon-ganh-ga-k...,2024-04-18,VSC,Viconship mua lại toàn bộ phần vốn góp của Xăn...,neutral
7,https://cafef.vn/doanh-nghiep-ti-hon-ganh-ga-k...,2024-04-18,Viconship,Thông tư 39/2023/TT-BGTVT được thông qua nâng ...,positive


In [ ]:
from azure.storage.blob import BlobServiceClient
import io
writer1 = io.BytesIO()
stock_filtered_df.to_excel(writer1,index=False)
file_name = f"new_stock.xlsx"
# changepm_file_name = f"changepm_project_lists_{suffix}.xlsx"
storge_connect_str="{connection_string}"
blob_service_client = BlobServiceClient.from_connection_string(storge_connect_str)
blob_client1 =blob_service_client.get_blob_client(container="hr-source", blob = "temp_folder/" + file_name)
blob_client1.upload_blob(writer1.getvalue(),overwrite=True)

{'etag': '"0x8DC605E482EF407"',
 'last_modified': datetime.datetime(2024, 4, 19, 10, 48, 43, tzinfo=datetime.timezone.utc),
 'content_md5': bytearray(b'\xe9\xe9\xd2\xc9\x82Q[\x94;F\x7f\xb4wl\xf3F'),
 'client_request_id': '63e828da-fe3a-11ee-b1b9-00163e73b5e7',
 'request_id': 'f626e2dd-601e-0052-2947-922a6e000000',
 'version': '2022-11-02',
 'version_id': None,
 'date': datetime.datetime(2024, 4, 19, 10, 48, 43, tzinfo=datetime.timezone.utc),
 'request_server_encrypted': True,
 'encryption_key_sha256': None,
 'encryption_scope': None}

In [ ]:
if len(stock_filtered_df) > 0:
    dbutils.notebook.run("/SA/Model2CosmosDB", 0)
else:
    print("Không có dữ liệu!")